<a href="https://colab.research.google.com/github/zhangshuo98767/AI/blob/main/A3T-GCN_TGCN_LSTM_GRU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
#install pytorch and all related libraries
#it will take some time

!pip install torch-scatter -f https://pytorch-geometric.com/whl/torch-${pt_version}.html
!pip install torch-sparse -f https://pytorch-geometric.com/whl/torch-${pt_version}.html
!pip install torch-cluster -f https://pytorch-geometric.com/whl/torch-${pt_version}.html
!pip install torch-spline-conv -f https://pytorch-geometric.com/whl/torch-${pt_version}.html
!pip install torch-geometric
!pip install torch-geometric-temporal
!pip install keras
!pip install --upgrade torch-scatter torch-sparse torch-cluster torch-spline-conv torch-geometric


Looking in links: https://pytorch-geometric.com/whl/torch-.html
Looking in links: https://pytorch-geometric.com/whl/torch-.html
Looking in links: https://pytorch-geometric.com/whl/torch-.html
Looking in links: https://pytorch-geometric.com/whl/torch-.html


In [8]:
# import all required libraries

import pandas as pd
import numpy as np
import torch
import os
import tensorflow as tf
import random
from tensorflow import keras
from keras.layers import LSTM, GRU, Dense
from keras.models import Sequential
from torch_geometric.utils import dense_to_sparse
from torch_geometric_temporal.nn.recurrent import A3TGCN, TGCN2
from torch_geometric_temporal.signal import StaticGraphTemporalSignalBatch
import torch.nn.functional as F
from sklearn.metrics import mean_squared_error,  mean_absolute_error
import scipy.stats


In [10]:
#Creating adjacency matrix

data_adj= pd.read_csv('/content/distanceNodes.txt', encoding='utf8', delimiter='\t')

#create new column calling Distance_KM, which calculates distance between nodes in km
data_adj['Distance_KM']=data_adj['NEAR_DIST']/1000

#  Change the nodes id
replacement_mapping_dict = {
    0: 141, 1: 143, 2: 195, 3: 181, 4: 5, 5: 88, 6: 138, 7: 254, 8: 142, 9: 113, 10: 192, 11: 243, 12: 78, 13: 92,
    14: 177, 15: 126, 16: 228, 17: 47, 18: 220, 19: 72, 20: 281, 21: 323, 22:217, 23:296
}
data_fin = data_adj[["IN_FID", "NEAR_FID"]].replace(replacement_mapping_dict)

# Replace columns with the new created columns
data_adj['IN_FID'] = data_fin["IN_FID"]
data_adj['NEAR_FID'] = data_fin["NEAR_FID"]

### create Adjacency matrix and replace column and index names according to grid cells id where air quality stations are located

am = pd.DataFrame(np.zeros(shape=(24, 24)))
am.rename(columns=replacement_mapping_dict, index =replacement_mapping_dict, inplace=True)
adj_mat = am.sort_index(axis=1)
adj_mat_complete = adj_mat.sort_index()


### Adjacency matrix

for i in data_adj.IN_FID.unique():
  for j in data_adj.NEAR_FID.unique():
    if i==j:
      adj_mat_complete.at[i,j]=0
    else:
      adj_mat_complete.at[i,j]=1/data_adj.loc[(data_adj['IN_FID'] == i) & (data_adj['NEAR_FID'] == j)]['Distance_KM']


adj_mat_complete

,5,47,72,78,88,92,113,126,138,141,...,192,195,217,220,228,243,254,281,296,323
5,0.000000,0.110356,0.236273,0.143935,0.181212,0.164228,0.112551,0.126298,0.117328,0.117353,...,0.091417,0.083470,0.066397,0.058654,0.075098,0.068696,0.054583,0.059056,0.058305,0.050814
47,0.110356,0.000000,0.109257,0.267139,0.095718,0.144390,0.208583,0.132625,0.079494,0.097987,...,0.087746,0.096880,0.102601,0.097302,0.082217,0.069394,0.084349,0.071076,0.064031,0.047170
72,0.236273,0.109257,0.000000,0.174832,0.645102,0.325035,0.148464,0.222798,0.221339,0.229932,...,0.147244,0.122364,0.082703,0.068796,0.107523,0.096458,0.064189,0.076388,0.076561,0.064459
78,0.143935,0.267139,0.174832,0.000000,0.145813,0.313177,0.442175,0.254499,0.113106,0.153721,...,0.126098,0.135770,0.116351,0.097973,0.109166,0.089090,0.086394,0.084737,0.077548,0.056153
88,0.181212,0.095718,0.645102,0.145813,0.000000,0.260419,0.133522,0.211733,0.330050,0.275207,...,0.166278,0.128027,0.081739,0.067254,0.115331,0.106098,0.063360,0.079434,0.081462,0.070607
92,0.164228,0.144390,0.325035,0.313177,0.260419,0.000000,0.271064,0.540075,0.176855,0.278847,...,0.180396,0.167196,0.109029,0.086970,0.133413,0.109384,0.079381,0.091725,0.087882,0.065522
113,0.112551,0.208583,0.148464,0.442175,0.133522,0.271064,0.000000,0.308841,0.114254,0.168040,...,0.149161,0.180343,0.157361,0.122457,0.135411,0.103990,0.105806,0.103478,0.091317,0.060882
126,0.126298,0.132625,0.222798,0.254499,0.211733,0.540075,0.308841,0.000000,0.180816,0.365729,...,0.247734,0.241840,0.131396,0.098479,0.175335,0.133346,0.089911,0.110489,0.104375,0.071900
138,0.117328,0.079494,0.221339,0.113106,0.330050,0.176855,0.114254,0.180816,0.000000,0.331141,...,0.223056,0.144352,0.083149,0.066961,0.139938,0.138718,0.064239,0.090053,0.097515,0.089265
141,0.117353,0.097987,0.229932,0.153721,0.275207,0.278847,0.168040,0.365729,0.331141,0.000000,...,0.409101,0.234668,0.109792,0.083599,0.198333,0.165686,0.078932,0.111642,0.114614,0.085327


In [11]:
# set the seed in order to provide reproducibility of the code

rnd_seed = 11

def set_seed(seed_num) -> None:
  random.seed(seed_num)
  np.random.seed(seed_num)
  tf.random.set_seed(seed_num)
  tf.experimental.numpy.random.seed(seed_num)
  tf.random.set_seed(seed_num)
  torch.manual_seed(seed_num)
  torch.cuda.manual_seed(seed_num)
  torch.backends.cudnn.deterministic = True
  torch.backends.cudnn.benchmark = False
  # When running on the CuDNN backend, two further options must be set
  os.environ['TF_CUDNN_DETERMINISTIC'] = '1'
  os.environ['TF_DETERMINISTIC_OPS'] = '1'
  # Set a fixed value for the hash seed
  os.environ["PYTHONHASHSEED"] = str(seed_num)
  print(f"Random seed set as {seed_num}")


In [12]:
# read the file containing nodes (air quality monitoring stations) features as pandas dataframe
Mad_data_2019 = pd.read_csv('/content/Mad_Station_2019.csv')
Mad_data_2022 = pd.read_csv('/content/Mad_Station_2022.csv')

# to delete 'windDir' column as we will not need it for further analysis.
Mad_data_2019 = Mad_data_2019.drop(['windDir'], axis=1)
Mad_data_2022 = Mad_data_2022.drop(['windDir'], axis=1)


In [14]:
# convert dataframes to numpy arrays

import numpy as np

# 先将DataFrame转换为一维数组
data_arr = Mad_data_2019.to_numpy().flatten()
data_arr_test = Mad_data_2022.to_numpy().flatten()

# 计算可以整除432的最大长度
n_total = data_arr.size // 432 * 432
n_total_test = data_arr_test.size // 432 * 432

# 截取整除部分
data_trimmed = data_arr[:n_total]
data_trimmed_test = data_arr_test[:n_total_test]

# 重塑为(-1, 24, 18)
fin_data = data_trimmed.reshape(-1, 24, 18)
fin_data_test = data_trimmed_test.reshape(-1, 24, 18)

print(fin_data.shape)
print(fin_data_test.shape)


(2476, 24, 18)
(2497, 24, 18)


In [15]:
# the function to return data in original scale (reversing Z score)

def reverse_zscore(pandas_series, mean, std):
    '''Mean and standard deviation should be of original variable before standardisation'''
    yis=pandas_series*std+mean
    return yis



Create the models and run them (A3T-GCN, TGCN, LSTM and GRU)

A3T-GCN


In [16]:
#to convert adjacency dataframe to numpy
adj = adj_mat_complete.to_numpy()

# standardise train data

data = fin_data.transpose(
            (1, 2, 0)
        )
data = data.astype(np.float32)

# standardise (via Z-Score Method)
means = np.mean(data, axis=(0, 2))
data_norm= data-means.reshape(1, -1, 1)
stds = np.std(data_norm, axis=(0, 2))
data_norm= data_norm/ stds.reshape(1, -1, 1)

#to convert adjacency matrix and standardised train data to torch
adj = torch.from_numpy(adj)
data_norm= torch.from_numpy(data_norm)


In [17]:
# standardise test data using means and standard deviation of train set

data_test = fin_data_test.transpose(
            (1, 2, 0)
        )
data_test = data_test.astype(np.float32)
data_test_norm= data_test- means.reshape(1, -1, 1)
data_test_norm= data_test_norm/ stds.reshape(1, -1, 1)

#to convert standardised test data to torch
data_test_norm = torch.from_numpy(data_test_norm)

In [18]:
adj.shape

torch.Size([24, 24])

In [19]:
data_test_norm.shape

torch.Size([24, 18, 2497])

In [20]:
#from adjacency matrix extract the edge indices and edge weights

edge_indices, values = dense_to_sparse(adj)
edge_indices = edge_indices.numpy()
values = values.numpy()
edges = edge_indices
edge_weights = values
batch =64

In [21]:
edges.shape

(2, 552)

In [22]:
edge_weights.shape

(552,)

In [26]:
class MadridDatasetLoader(object):
    """The dataset is based on 24 stations (nodes) each having 18 features (nodal features)
    and 276 edges connecting each pair of nodes, the edge weights are the distance between the edges.
    """

    def __init__(self, data_norm, edges, edge_weights, batch):
        super(MadridDatasetLoader, self).__init__()

        self.data_norm = data_norm
        self.edges = edges
        self.edge_weights= edge_weights
        self.batch = batch


    def _generate_task(self, num_timesteps_in: int = 6, num_timesteps_out: int = 6):
        """Uses the node features of the graph and generates a feature/target
        relationship of the shape
        (num_nodes, num_node_features, num_timesteps_in) -> (num_nodes, num_timesteps_out)


        Args:
            num_timesteps_in (int): number of timesteps the sequence model sees
            num_timesteps_out (int): number of timesteps the sequence model has to predict
        """
        time_steps_starter =   0 # it can be assigned as one of the following {0, 12, 24, 36}
        indices = [
            (i, i +time_steps_starter+ (num_timesteps_in + num_timesteps_out))
            for i in range(self.data_norm.shape[2] - (time_steps_starter+num_timesteps_in + num_timesteps_out) + 1)
        ]
        print(indices)
        # Generate observations
        features, target = [], []
        for i, j in indices:
            features.append((self.data_norm[:, :, i : i + num_timesteps_in]).numpy())
            target.append((self.data_norm[  :, 0, i + num_timesteps_in +time_steps_starter: j]).numpy())

        self.features = features
        self.targets = target

    def get_dataset(
            self, num_timesteps_in: int = 6, num_timesteps_out: int = 6
    ) -> StaticGraphTemporalSignalBatch:
        """Returns data iterator for the dataset as an instance of the
        static graph temporal signal class.

        Return types:
            * **dataset** *(StaticGraphTemporalSignalBatch)* - The forecasting dataset.
        """

        self._generate_task(num_timesteps_in, num_timesteps_out)
        dataset = StaticGraphTemporalSignalBatch(
            self.edges, self.edge_weights, self.features, self.targets, self.batch
        )

        return dataset

In [27]:
loader = MadridDatasetLoader(data_test_norm, edges, edge_weights, batch)
dataset_test = loader.get_dataset(num_timesteps_in=12, num_timesteps_out=12)
print("Dataset type:  ", dataset_test)


[(0, 24), (1, 25), (2, 26), (3, 27), (4, 28), (5, 29), (6, 30), (7, 31), (8, 32), (9, 33), (10, 34), (11, 35), (12, 36), (13, 37), (14, 38), (15, 39), (16, 40), (17, 41), (18, 42), (19, 43), (20, 44), (21, 45), (22, 46), (23, 47), (24, 48), (25, 49), (26, 50), (27, 51), (28, 52), (29, 53), (30, 54), (31, 55), (32, 56), (33, 57), (34, 58), (35, 59), (36, 60), (37, 61), (38, 62), (39, 63), (40, 64), (41, 65), (42, 66), (43, 67), (44, 68), (45, 69), (46, 70), (47, 71), (48, 72), (49, 73), (50, 74), (51, 75), (52, 76), (53, 77), (54, 78), (55, 79), (56, 80), (57, 81), (58, 82), (59, 83), (60, 84), (61, 85), (62, 86), (63, 87), (64, 88), (65, 89), (66, 90), (67, 91), (68, 92), (69, 93), (70, 94), (71, 95), (72, 96), (73, 97), (74, 98), (75, 99), (76, 100), (77, 101), (78, 102), (79, 103), (80, 104), (81, 105), (82, 106), (83, 107), (84, 108), (85, 109), (86, 110), (87, 111), (88, 112), (89, 113), (90, 114), (91, 115), (92, 116), (93, 117), (94, 118), (95, 119), (96, 120), (97, 121), (98, 12

In [28]:
loader = MadridDatasetLoader(data_norm, edges, edge_weights, batch)
dataset = loader.get_dataset(num_timesteps_in=12, num_timesteps_out=12)
print("Dataset type:  ", dataset)


[(0, 24), (1, 25), (2, 26), (3, 27), (4, 28), (5, 29), (6, 30), (7, 31), (8, 32), (9, 33), (10, 34), (11, 35), (12, 36), (13, 37), (14, 38), (15, 39), (16, 40), (17, 41), (18, 42), (19, 43), (20, 44), (21, 45), (22, 46), (23, 47), (24, 48), (25, 49), (26, 50), (27, 51), (28, 52), (29, 53), (30, 54), (31, 55), (32, 56), (33, 57), (34, 58), (35, 59), (36, 60), (37, 61), (38, 62), (39, 63), (40, 64), (41, 65), (42, 66), (43, 67), (44, 68), (45, 69), (46, 70), (47, 71), (48, 72), (49, 73), (50, 74), (51, 75), (52, 76), (53, 77), (54, 78), (55, 79), (56, 80), (57, 81), (58, 82), (59, 83), (60, 84), (61, 85), (62, 86), (63, 87), (64, 88), (65, 89), (66, 90), (67, 91), (68, 92), (69, 93), (70, 94), (71, 95), (72, 96), (73, 97), (74, 98), (75, 99), (76, 100), (77, 101), (78, 102), (79, 103), (80, 104), (81, 105), (82, 106), (83, 107), (84, 108), (85, 109), (86, 110), (87, 111), (88, 112), (89, 113), (90, 114), (91, 115), (92, 116), (93, 117), (94, 118), (95, 119), (96, 120), (97, 121), (98, 12

In [29]:
# define A3T-GCN

with tf.device('/device:GPU:0'):
  set_seed(rnd_seed)

  unit_num  = 256  # This is number of units in order to construct the architecture of the A3T-GCN.

  class TemporalGNN(torch.nn.Module):
      def __init__(self, node_features, periods):
          super(TemporalGNN, self).__init__()
          # Attention Temporal Graph Convolutional Cell
          self.tgnn = A3TGCN(in_channels=node_features,
                            out_channels=unit_num,
                            periods=periods)
          # Equals single-shot prediction
          self.linear = torch.nn.Linear(unit_num, periods)

      def forward(self, x, edge_index,  edge_weight):
          """
          x = Node features for T time steps
          edge_index = Graph edge indices
          """
          h = self.tgnn(x, edge_index,  edge_weight)
          h = F.relu(h)
          h = self.linear(h)
          return h

TemporalGNN(node_features=18, periods=12)

Random seed set as 11


TemporalGNN(
  (tgnn): A3TGCN(
    (_base_tgcn): TGCN(
      (conv_z): GCNConv(18, 256)
      (linear_z): Linear(in_features=512, out_features=256, bias=True)
      (conv_r): GCNConv(18, 256)
      (linear_r): Linear(in_features=512, out_features=256, bias=True)
      (conv_h): GCNConv(18, 256)
      (linear_h): Linear(in_features=512, out_features=256, bias=True)
    )
  )
  (linear): Linear(in_features=256, out_features=12, bias=True)
)

In [30]:
# GPU support
device = torch.device('cuda') # cuda


# Create model and optimizers
model = TemporalGNN(node_features=18, periods=12).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
model.train()

print("Running training...")
for epoch in range(100):
    loss = 0
    step = 0
    for snapshot in dataset:
        snapshot = snapshot.to(device)
        # Get model predictions
        y_hat = model(snapshot.x, snapshot.edge_index, snapshot.edge_attr)
        # Mean Squared Error
        loss = loss + torch.mean((y_hat-snapshot.y)**2)
        step += 1


    loss = loss / (step + 1)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    print("Epoch {} train MSE: {:.4f}".format(epoch, loss.item()))

AssertionError: Torch not compiled with CUDA enabled

In [31]:
model.eval()
loss = 0
step = 0

# Store for analysis
predictions = []
labels = []

for snapshot in dataset_test:
    snapshot = snapshot.to(device)
    # Get predictions
    y_hat = model(snapshot.x, snapshot.edge_index, snapshot.edge_attr)
    yhat_reverse = reverse_zscore(y_hat, means[0], stds[0])
    snapshot.y_reverse = reverse_zscore(snapshot.y, means[0], stds[0])
    # Root Mean Squared Error
    loss = loss + torch.sqrt(torch.mean((yhat_reverse-snapshot.y_reverse)**2))
    # Store for analysis below
    labels.append(snapshot.y_reverse)
    predictions.append(yhat_reverse)
    step += 1

loss = loss / (step+1)
loss = loss.item()
print("Test RMSE: {:.4f}".format(loss))

NameError: name 'model' is not defined

In [ ]:
model.eval()
loss = 0
step = 0

# Store for analysis
predictions = []
labels = []

for snapshot in dataset_test:
    snapshot = snapshot.to(device)
    # Get predictions
    y_hat = model(snapshot.x, snapshot.edge_index, snapshot.edge_attr)
    yhat_reverse = reverse_zscore(y_hat, means[0], stds[0])
    snapshot.y_reverse = reverse_zscore(snapshot.y, means[0], stds[0])
    # Mean Absolute Error
    loss = loss + torch.mean(torch.abs(yhat_reverse-snapshot.y_reverse))
    # Store for analysis below
    labels.append(snapshot.y_reverse)
    predictions.append(yhat_reverse)
    step += 1


loss = loss / (step+1)
loss = loss.item()
print("Test MAE: {:.4f}".format(loss))

In [ ]:
# it is calculated for all nodes

ALLNode_pred = []
ALLNode_true = []
for item in predictions:
  for node in range(24):
    for hour in range(12):
      ALLNode_pred.append(item[node][hour].detach().cpu().numpy().item(0))


for item in labels:
  for node in range(24):
    for hour in range(12):
      ALLNode_true.append(item[node][hour].detach().cpu().numpy().item(0))


ALLNode_pred_np = np.array(ALLNode_pred)
ALLNode_pred_np_resh = ALLNode_pred_np.reshape(-1, 24, 12)
ALLNode_true_np = np.array(ALLNode_true)
ALLNode_true_np_resh = ALLNode_true_np.reshape(-1, 24, 12)


scipy.stats.pearsonr(ALLNode_pred_np, ALLNode_true_np)

TGCN

In [ ]:
#to convert adjacency dataframe to numpy
adj = adj_mat_complete.to_numpy()

# standardise train data

data = fin_data.transpose(
            (1, 2, 0)
        )
data = data.astype(np.float32)

# standardise (via Z-Score Method)
means = np.mean(data, axis=(0, 2))
data_norm= data-means.reshape(1, -1, 1)
stds = np.std(data_norm, axis=(0, 2))
data_norm= data_norm/ stds.reshape(1, -1, 1)

#to convert adjacency matrix and standardised train data to torch
adj = torch.from_numpy(adj)
data_norm= torch.from_numpy(data_norm)


In [ ]:
# standardise test data using means and standard deviation of train set

data_test = fin_data_test.transpose(
            (1, 2, 0)
        )
data_test = data_test.astype(np.float32)
data_test_norm= data_test- means.reshape(1, -1, 1)
data_test_norm= data_test_norm/ stds.reshape(1, -1, 1)

#to convert standardised test data to torch
data_test_norm = torch.from_numpy(data_test_norm)

In [ ]:
adj.shape

torch.Size([24, 24])

In [ ]:
data_test_norm.shape

torch.Size([24, 18, 4343])

In [ ]:
#from adjacency matrix extract the edge indices and edge weights

edge_indices, values = dense_to_sparse(adj)
edge_indices = edge_indices.numpy()
values = values.numpy()
edges = edge_indices
edge_weights = values
batch =64

In [ ]:
edges.shape

(2, 552)

In [ ]:
edge_weights.shape

(552,)

In [ ]:
class MadridDatasetLoader(object):
    """The dataset is based on 24 stations (nodes) each having 18 features (nodal features)
    and 276 edges connecting each pair of nodes, the edge weights are the distance between the edges.
    """

    def __init__(self, data_norm, edges, edge_weights, batch):
        super(MadridDatasetLoader, self).__init__()

        self.data_norm = data_norm
        self.edges = edges
        self.edge_weights= edge_weights
        self.batch = batch


    def _generate_task(self, num_timesteps_in: int = 6, num_timesteps_out: int = 6):
        """Uses the node features of the graph and generates a feature/target
        relationship of the shape
        (num_nodes, num_node_features, num_timesteps_in) -> (num_nodes, num_timesteps_out)


        Args:
            num_timesteps_in (int): number of timesteps the sequence model sees
            num_timesteps_out (int): number of timesteps the sequence model has to predict
        """
        time_steps_starter =   36 # it can be assigned as one of the following {0, 12, 24, 36}
        indices = [
            (i, i +time_steps_starter+ (num_timesteps_in + num_timesteps_out))
            for i in range(self.data_norm.shape[2] - (time_steps_starter+num_timesteps_in + num_timesteps_out) + 1)
        ]
        print(indices)
        # Generate observations
        features, target = [], []
        for i, j in indices:
            features.append((self.data_norm[:, :, i : i + num_timesteps_in]).numpy().transpose(
            (2, 0, 1)
        ))
            target.append((self.data_norm[  :, 0, i + num_timesteps_in +time_steps_starter: j]).numpy())

        self.features = features
        self.targets = target

    def get_dataset(
        self, num_timesteps_in: int = 6, num_timesteps_out: int = 6
    ) -> StaticGraphTemporalSignal:
        """Returns data iterator for the dataset as an instance of the
        static graph temporal signal class.

        Return types:
            * **dataset** *(StaticGraphTemporalSignal)* - The forecasting dataset.
        """

        self._generate_task(num_timesteps_in, num_timesteps_out)
        dataset = StaticGraphTemporalSignalBatch(
            self.edges, self.edge_weights, self.features, self.targets, self.batch
        )

        return dataset

In [ ]:
loader = MadridDatasetLoader(data_test_norm, edges, edge_weights, batch)
dataset_test = loader.get_dataset(num_timesteps_in=12, num_timesteps_out=12)
print("Dataset type:  ", dataset_test)


In [ ]:
loader = MadridDatasetLoader(data_norm, edges, edge_weights, batch)
dataset = loader.get_dataset(num_timesteps_in=12, num_timesteps_out=12)
print("Dataset type:  ", dataset)


In [ ]:
next(iter(dataset))

In [ ]:
# define A3T-GCN

with tf.device('/device:GPU:0'):
  set_seed(rnd_seed)

  unit_num  = 256 # This is number of units in order to construct the architecture of the TGCN.
  class TemporalGNN(torch.nn.Module):
      def __init__(self, node_features):
          super(TemporalGNN, self).__init__()
          # Temporal Graph Convolutional Cell
          self.tgnn = TGCN2(in_channels=node_features,
                            out_channels=unit_num, batch_size=12)
          # Equals single-shot prediction
          self.linear = torch.nn.Linear(unit_num, 12)

      def forward(self, x, edge_index,  edge_weight):
          """
          x = Node features for T time steps
          edge_index = Graph edge indices
          """
          h = self.tgnn(x, edge_index,  edge_weight)
          h = F.relu(h)
          h = self.linear(h)
          return h

TemporalGNN(node_features=18)

Random seed set as 11


TemporalGNN(
  (tgnn): TGCN2(
    (conv_z): GCNConv(18, 256)
    (linear_z): Linear(in_features=512, out_features=256, bias=True)
    (conv_r): GCNConv(18, 256)
    (linear_r): Linear(in_features=512, out_features=256, bias=True)
    (conv_h): GCNConv(18, 256)
    (linear_h): Linear(in_features=512, out_features=256, bias=True)
  )
  (linear): Linear(in_features=256, out_features=12, bias=True)
)

In [ ]:
# GPU support
device = torch.device('cpu') # cuda


# Create model and optimizers
model = TemporalGNN(node_features=18).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
model.train()

print("Running training...")
for epoch in range(100):
    loss = 0
    step = 0
    for snapshot in dataset:
        snapshot = snapshot.to(device)
        # Get model predictions
        y_hat = model(snapshot.x, snapshot.edge_index, snapshot.edge_attr)
        # Mean Squared Error
        loss = loss + torch.mean((y_hat-snapshot.y)**2)
        step += 1


    loss = loss / (step + 1)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    print("Epoch {} train MSE: {:.4f}".format(epoch, loss.item()))

Running training...
Epoch 0 train MSE: 0.9955


In [ ]:
# it is calculated for all nodes

ALLNode_pred = []
ALLNode_true = []
for item in predictions:
  for batch in range(12):
    for node in range(24):
      for hour in range(12):
        ALLNode_pred.append(item[batch][node][hour].detach().cpu().numpy().item(0))


for item in labels:
  for node in range(24):
    for hour in range(12):
      ALLNode_true.append(item[node][hour].detach().cpu().numpy().item(0))


ALLNode_pred_np = np.array(ALLNode_pred)
ALLNode_pred_np_resh = ALLNode_pred_np.reshape(-1, 12, 24, 12)
ALLNode_true_np = np.array(ALLNode_true)
ALLNode_true_np_resh = ALLNode_true_np.reshape(-1, 24, 12)

In [ ]:
model.eval()
loss = 0
step = 0

# Store for analysis
predictions = []
labels = []

for snapshot in dataset_test:
    snapshot = snapshot.to(device)
    # Get predictions
    y_hat = model(snapshot.x, snapshot.edge_index, snapshot.edge_attr)
    yhat_reverse = reverse_zscore(y_hat, means[0], stds[0])
    snapshot.y_reverse = reverse_zscore(snapshot.y, means[0], stds[0])
    # Mean Absolute Error
    loss = loss + torch.mean(torch.abs(yhat_reverse-snapshot.y_reverse))
    # Store for analysis below
    labels.append(snapshot.y_reverse)
    predictions.append(yhat_reverse)
    step += 1


loss = loss / (step+1)
loss = loss.item()
print("Test MAE: {:.4f}".format(loss))

In [ ]:
ALLNode_pred_np_resh_Update = np.mean(ALLNode_pred_np_resh, axis=1)
true = ALLNode_true_np_resh.reshape(-1)
pred = ALLNode_pred_np_resh_Update.reshape(-1)
rmse = mean_squared_error(true, pred, squared=False)
mae = mean_absolute_error(true, pred)
scipy.stats.pearsonr(true, pred)

LSTM

In [ ]:
# standardise train data

data = fin_data.transpose(
            (1, 2, 0)
        )
data = data.astype(np.float32)

# standardise (via Z-Score Method)
means = np.mean(data, axis=(0, 2))
data_norm= data-means.reshape(1, -1, 1)
stds = np.std(data_norm, axis=(0, 2))
data_norm= data_norm/ stds.reshape(1, -1, 1)
fin_data_norm = data_norm.transpose(2, 0, 1)



In [ ]:
fin_data.shape

(4344, 24, 18)

In [ ]:
# standardise test data

data_test = fin_data_test.transpose(
            (1, 2, 0)
        )
data_test = data_test.astype(np.float32)
data_test_norm= data_test- means.reshape(1, -1, 1)
data_test_norm= data_test_norm/ stds.reshape(1, -1, 1)
fin_data_test_norm = data_test_norm.transpose(2, 0, 1)


In [ ]:
fin_data_test_norm.shape

(4343, 24, 18)

In [ ]:
# split dataset to X and y (dependent and independent)

def split_sequence(sequence, seq_notNorm, time_steps):
	X, y = list(), list()
	for i in range(len(sequence)):

		# find the end of this pattern
		end_ix = i +12
		time_steps_starter = 0 # it can be assigned as one of the following {0, 12, 24, 36}

		# check if we are beyond the sequence
		if end_ix+time_steps_starter+time_steps > len(sequence)-1:
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequence[i:end_ix], seq_notNorm[end_ix+time_steps_starter:end_ix+time_steps_starter+time_steps]
		X.append(seq_x)
		y.append(seq_y)

	return np.array(X), np.array(y)


# choose a number of time steps
time_steps = 12
X_train, y_train = split_sequence(fin_data_norm, fin_data_norm, time_steps)
X_test, y_test = split_sequence(fin_data_test_norm, fin_data_test_norm, time_steps)

# to select only nitrogen dioxide as a target feature
y_train = y_train[:, :, :, 0]
y_test = y_test[:, :, :, 0]


In [ ]:
y_test.shape

(4319, 12, 24)

In [ ]:
X_train.shape

(4320, 12, 24, 18)

In [ ]:
# reshape the data for LSTM input

number_selected_columns =18
X_train_reshaped = X_train.reshape((X_train.shape[0], X_train.shape[1], 24*number_selected_columns))
X_test_reshaped = X_test.reshape((X_test.shape[0], X_test.shape[1], 24*number_selected_columns))


In [ ]:
# define model

with tf.device('/device:GPU:0'):
  set_seed(rnd_seed)
  model = Sequential()
  model.add(Dense(432,input_shape=(X_train.shape[1], 24*number_selected_columns)))
  model.add(LSTM(512, return_sequences=True))
  model.add(Dense(24))
  model.compile(optimizer='adam', loss='mse')

Random seed set as 11


In [ ]:
# run LSTM

lstm_model = model.fit(X_train_reshaped, y_train, epochs=100, verbose=2)


In [ ]:
y_test.shape

(4319, 12, 24)

In [ ]:
yhat = model.predict(X_test_reshaped, verbose=1)

135/135 [==============================] - 2s 7ms/step


In [ ]:
yhat_reverse = reverse_zscore(yhat, means[0], stds[0])
ytest_reverse = reverse_zscore(y_test, means[0], stds[0])
yhat_reshaped = yhat_reverse.reshape(-1,24)
y_test_reshaped= ytest_reverse.reshape(-1,24)
rmse = mean_squared_error(yhat_reshaped, y_test_reshaped, squared=False)
mae = mean_absolute_error(yhat_reshaped, y_test_reshaped)
print('Test Score: %.2f RMSE' % (rmse))
print('Test Score: %.2f MAE' % (mae))

In [ ]:
scipy.stats.pearsonr(yhat_reshaped.reshape(-1), y_test_reshaped.reshape(-1))

GRU

In [ ]:
# standardise train data

data = fin_data.transpose(
            (1, 2, 0)
        )
data = data.astype(np.float32)

# standardise (via Z-Score Method)
means = np.mean(data, axis=(0, 2))
data_norm= data-means.reshape(1, -1, 1)
stds = np.std(data_norm, axis=(0, 2))
data_norm= data_norm/ stds.reshape(1, -1, 1)
fin_data_norm = data_norm.transpose(2, 0, 1)



In [ ]:
fin_data.shape

(4344, 24, 18)

In [ ]:
# standardise test data

data_test = fin_data_test.transpose(
            (1, 2, 0)
        )
data_test = data_test.astype(np.float32)
data_test_norm= data_test- means.reshape(1, -1, 1)
data_test_norm= data_test_norm/ stds.reshape(1, -1, 1)
fin_data_test_norm = data_test_norm.transpose(2, 0, 1)


In [ ]:
fin_data_test_norm.shape

(4343, 24, 18)

In [ ]:
# split dataset to X and y (dependent and independent)

def split_sequence(sequence, seq_notNorm, time_steps):
	X, y = list(), list()
	for i in range(len(sequence)):

		# find the end of this pattern
		end_ix = i +12
		time_steps_starter = 0 # it can be assigned as one of the following {0, 12, 24, 36}

		# check if we are beyond the sequence
		if end_ix+time_steps_starter+time_steps > len(sequence)-1:
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequence[i:end_ix], seq_notNorm[end_ix+time_steps_starter:end_ix+time_steps_starter+time_steps]
		X.append(seq_x)
		y.append(seq_y)

	return np.array(X), np.array(y)


# choose a number of time steps
time_steps = 12
X_train, y_train = split_sequence(fin_data_norm, fin_data_norm, time_steps)
X_test, y_test = split_sequence(fin_data_test_norm, fin_data_test_norm, time_steps)

# to select only nitrogen dioxide as a target feature
y_train = y_train[:, :, :, 0]
y_test = y_test[:, :, :, 0]


In [ ]:
y_test.shape

(4319, 12, 24)

In [ ]:
X_train.shape

(4320, 12, 24, 18)

In [ ]:

# reshape the data for GRU input

number_selected_columns =18
X_train_reshaped = X_train.reshape((X_train.shape[0], X_train.shape[1], 24*number_selected_columns))
X_test_reshaped = X_test.reshape((X_test.shape[0], X_test.shape[1], 24*number_selected_columns))


In [ ]:
# define model

with tf.device('/device:GPU:0'):
  set_seed(rnd_seed)
  model = Sequential()
  model.add(Dense(432,input_shape=(X_train.shape[1], 24*number_selected_columns)))
  model.add(GRU(512, return_sequences=True))
  model.add(Dense(24))
  model.compile(optimizer='adam', loss='mse')

Random seed set as 11


In [ ]:
# run GRU

gru_model = model.fit(X_train_reshaped, y_train, epochs=100, verbose=2)


In [ ]:
yhat = model.predict(X_test_reshaped, verbose=1)

135/135 [==============================] - 4s 26ms/step


In [ ]:
yhat_reverse = reverse_zscore(yhat, means[0], stds[0])
ytest_reverse = reverse_zscore(y_test, means[0], stds[0])
yhat_reshaped = yhat_reverse.reshape(-1,24)
y_test_reshaped= ytest_reverse.reshape(-1,24)
rmse = mean_squared_error(yhat_reshaped, y_test_reshaped, squared=False)
mae = mean_absolute_error(yhat_reshaped, y_test_reshaped)
print('Test Score: %.2f RMSE' % (rmse))
print('Test Score: %.2f MAE' % (mae))

In [ ]:
scipy.stats.pearsonr(yhat_reshaped.reshape(-1), y_test_reshaped.reshape(-1))